In [1]:
import os, errno
import json
import yaml
import argparse
import sys
import traceback
import time
import pandas as pd
import torch
import torch.utils.data as data_utils
import transformers
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# default constants

VBERT_MODEL = "vBERT-2020-Base"
CLASSIFICATION_LAYER_WIDTH = 768 if "Base" in VBERT_MODEL else 1024
MAX_STRING_LEN = 128

In [2]:
path_data_base = 'data'
name_testset = 'importance_detection_testset_200_12000.csv'
path_testset = os.path.join(path_data_base, name_testset)

In [3]:
path_model_base = 'models-instaml-finetune'
version_model = 'v1216'
name_file_config = '.model.json'
path_model = os.path.join(path_model_base, version_model)
path_config = os.path.join(path_model, name_file_config)

with open(path_config, 'r') as file:
    config_model = json.load(file)
# end

for k, v in config_model.get('bert').items():
    config_model[k] = v
# end

config_model['root'] = path_model

del config_model['allmetrics']
del config_model['metrics']

In [4]:
device = 'cpu'

In [5]:
class BertClassifier(torch.nn.Module):

    def __init__(self, config):
        super(BertClassifier, self).__init__()
        self.config = config
        self.model_dir = config.get("root", ".")
        self.model_name = config.get("name", VBERT_MODEL)
        self.classification_layer_width = config.get("input_size", CLASSIFICATION_LAYER_WIDTH)
        self.classes = config.get("classes", ["0", "1"])
        self.num_classes = config.get("output_size", len(self.classes))
        self.max_string_length = config.get("max_length", MAX_STRING_LEN)
        self.bert_config_file = self.model_dir + "/bert_config.json"
        self.model_file_path = self.model_dir + "/model.pt"
#        self.device = config.get("device", "cpu")
        self.device = 'cpu'
#         self.device = 'cuda'
        self.metrics = [ "accuracy", "precision", "recall", "F-score" ]

        # Load the vBERT vocabulary into the tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(self.model_dir)
    # end

    def load_bert_for_training(self):
        print('Loading vBERT model: ' + self.model_name)
        self.l1 = BertModel.from_pretrained(pretrained_model_name_or_path=self.config.bert_model_dir)
        print("Adding {}x{} classification layer".format(self.classification_layer_width, self.num_classes))
        self.classifier = torch.nn.Linear(self.classification_layer_width, self.num_classes)

    def load_bert_for_inference(self):
        print('Loading vBERT config')
        self.l1 = BertModel(BertConfig.from_pretrained(self.bert_config_file))
        print("Adding {}x{} classification layer".format(self.classification_layer_width, self.num_classes))
        self.classifier = torch.nn.Linear(self.classification_layer_width, self.num_classes)

    # Encode the input with vBERT, read output from the last layer of vBERT, and pass to the classification layer
    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
#         print(output_1.keys())
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0,:]
#         pooler = hidden_state[:, 0]
#         print(pooler.numpy()[0][:5])
        output = self.classifier(pooler)
        print(type(output))
        print(output)
        return output

#     def do_inference(self, data_loader, test_output=None):
#         model = self.eval()
#         predictions = []
#         real_values = []
#         with torch.no_grad():
#             for data in data_loader:
#                 input_ids = data['ids'].to(self.device)
#                 attention_mask = data['mask'].to(self.device)
#                 targets = data['targets'].to(self.device)
#                 outputs = model(input_ids, attention_mask)
#                 _, preds = torch.max(outputs, dim=1)
#                 predictions.extend(preds)
#                 real_values.extend(targets)
#                 if test_output != None:
#                     test_output.extend(outputs)
#         predictions = torch.stack(predictions).cpu()
#         real_values = torch.stack(real_values).cpu()

#         test_accu = 100 * accuracy_score(real_values, predictions)
#         test_precision, test_recall, test_fscore, ignore = precision_recall_fscore_support(real_values, predictions, average='macro')
#         test_precision *= 100
#         test_recall *= 100
#         test_fscore *= 100
#         metrics = [ test_accu, test_precision, test_recall, test_fscore ]
#         return predictions, real_values, metrics
#     # end
# end

In [6]:
class Prepare(Dataset):
    def __init__(self, dataframe, tokenizer, max_len, classes):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.classes = classes
        
        self.index_class_label = {klass:index for index, klass in enumerate(self.classes)}
    # end

    def __getitem__(self, index):
        title = str(self.data['log'][index])
        title = " ".join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        label = self.index_class_label[self.data['pcc'][index]]

#         target = torch.tensor(label, dtype=torch.long)
        target = label

        return {
            'ids': torch.tensor(ids, dtype=torch.long).to(device),
            'mask': torch.tensor(mask, dtype=torch.long).to(device),
            'targets': label
        }

    def __len__(self):
        return self.len
    # end
# end


In [7]:
from tqdm import tqdm

# def classify_text(model, txt, conf=None):
#     tokenizer = model.tokenizer
#     max_string_length = model.max_string_length
    
#     dataset = pd.DataFrame.from_csv(path_testset)
#     prepared_set = Prepare(dataset, tokenizer, max_string_length)
#     params = {'batch_size': 1,
#                 'shuffle': False,
#                 'num_workers': 0
#             }
#     loader = DataLoader(prepared_set, **params)
#     outputs = []
#     predictions, real_values, metrics = self.do_inference(loader, outputs)
#     classes = model.classes
#     result = { 'classes': [ ], 'text': txt, 'top_class': classes[int(predictions[0])], 'top_class_index': int(predictions[0]) }
#     xi = 0
#     for x in classes:
#         result['classes'].append({ 'class_name': x, 'confidence': float(outputs[0][xi]) })
#         xi += 1
#     return result


#     dataset_eval = Dataset(train_texts + valid_texts, train_labels + valid_labels)
#     dataload_eval = torch.utils.data.DataLoader(dataset_eval, batch_size=4, shuffle=True)

#     y_hats_all = []
#     y_true_all = []
#     for _valid_texts, _valid_labels in tqdm(dataload_eval):
#         outputs = predict_minibatch(model, _valid_texts)
#         y_hats = outputs['y_hats'].tolist()
#         y_true = _valid_labels.detach().cpu().numpy().tolist()   
#         y_hats_all += y_hats
#         y_true_all += y_true
#     # end

#     print('precision_score: {}'.format(precision_score(y_true_all, y_hats_all)))
#     print('recall_score: {}'.format(recall_score(y_true_all, y_hats_all)))
#     print('f1_score: {}'.format(f1_score(y_true_all, y_hats_all)))
# # end

In [8]:
df = pd.read_csv(path_testset)

In [9]:
df[:3]

,pc,pcc,log
0,comment,comment,timestamp 5776059 admission failure in path ho...
1,comment,comment,timestamp cpu 2 2156005 os fs os fs get mount ...
2,comment,comment,timestamp hex id debug id hex id vsan ctl get ...


In [10]:
model = BertClassifier(config_model)
model.load_bert_for_inference()
model.to(device)
model.load_state_dict(torch.load(model.model_file_path, map_location=torch.device(device)))

file models-instaml-finetune/v1216/config.json not found


Loading vBERT config
Adding 768x2 classification layer


<All keys matched successfully>

In [ ]:
dataset = pd.read_csv(path_testset)
max_length = model.max_string_length
classes = model.classes
tokenizer = model.tokenizer

prepared_set = Prepare(dataset, tokenizer, max_length, classes)
params_dataloader = {
    'batch_size': 1,
    'shuffle': False,
    'num_workers': 0
}
data_loader = DataLoader(prepared_set, **params_dataloader)
y_hats_all = []
y_true_all = []
ids_top3 = []

with torch.no_grad():
    for data in tqdm(data_loader):
#         ids_top3.append(data['ids'].cpu().numpy()[0][:3])
        input_ids = data['ids'].to(device)
        attention_mask = data['mask'].to(device)
        outputs = model(input_ids, attention_mask)
#         print(outputs)
        _, preds = torch.max(outputs, dim=1)
        y_hats_all.append(preds.cpu().numpy()[0])
        y_true_all.append(data['targets'][0])
    # end
# end

print('accuracy_score: {}'.format(accuracy_score(y_true_all, y_hats_all)))
print('precision_score: {}'.format(precision_score(y_true_all, y_hats_all, zero_division=1)))
print('recall_score: {}'.format(recall_score(y_true_all, y_hats_all, zero_division=1)))
print('f1_score: {}'.format(f1_score(y_true_all, y_hats_all, zero_division=1)))
# print(ids_top3)


  0%|          | 1/12546 [00:03<10:50:37,  3.11s/it]

<class 'torch.Tensor'>
tensor([[-2.1091,  1.9549]])


  0%|          | 2/12546 [00:06<10:38:25,  3.05s/it]

<class 'torch.Tensor'>
tensor([[ 3.0900, -2.9522]])


  0%|          | 3/12546 [00:08<9:55:05,  2.85s/it] 

...

  0%|          | 44/12546 [01:13<9:01:37,  2.60s/it]

<class 'torch.Tensor'>
tensor([[-1.0964,  0.9258]])


  0%|          | 45/12546 [01:16<10:11:07,  2.93s/it]

<class 'torch.Tensor'>
tensor([[-1.6309,  1.7456]])


  0%|          | 45/12546 [01:17<5:58:22,  1.72s/it] 


In [ ]:
len(y_hats_all)

In [ ]:
# with open('y_hats_all.txt', 'w+') as file:
#     file.write(json.dumps([int(x) for x in y_hats_all]))

In [ ]:
# with open('y_true_all.txt', 'w+') as file:
#     file.write(json.dumps([int(x) for x in y_true_all]))

In [ ]:
# print('accuracy_score: {}'.format(accuracy_score(y_true_all, y_hats_all)))
# print('precision_score: {}'.format(precision_score(y_true_all, y_hats_all, zero_division=1, average='macro')))
# print('recall_score: {}'.format(recall_score(y_true_all, y_hats_all, zero_division=1, average='macro')))
# print('f1_score: {}'.format(f1_score(y_true_all, y_hats_all, zero_division=1, average='macro')))